In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.layers import *
from tensorflow.keras.models import *

2023-04-03 11:02:41.894210: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 11:02:42.020505: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-03 11:02:42.047509: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a suppor

# U-Net

In [11]:
from tensorflow import keras

img_size=(224, 224);  n_class = 200;  batch_size=32

def U_Net(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3, ))
    
    # U-Net의 DownSampling
    x = Conv2D(32, 3, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # shortcut connection을 위해
    previous_block_activation = x
    
    for filters in [64, 128, 256]:
        x = Activation('relu')(x)
        x = SeparableConv2D(filters, 3, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = SeparableConv2D(filters, 3, padding='same')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(3, strides=2, padding='same')(x)
        
        residual = Conv2D(filters, 1, strides=2, padding='same')(previous_block_activation)
        x = add([x, residual])
        previous_block_activation = x
        
        
        
    # U-Net의 UpSampling
    for filters in [256, 128, 64, 32]:
        x = Activation('relu')(x)
        x = Conv2DTranspose(filters, 3, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2DTranspose(filters, 3, padding='same')(x)
        x = BatchNormalization()(x)
        x = UpSampling2D(2)(x)
        
        residual = UpSampling2D(2)(previous_block_activation)
        residual = Conv2D(filters, 1, padding='same')(residual)
        x = add([x, residual])
        previous_block_activation = x
        
        
        
        
    outputs = Conv2D(num_classes, 3, activation='softmax', padding='same')(x)
    model = Model(inputs, outputs)
    return model

model = U_Net(img_size, n_class)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_515 (Conv2D)            (None, 112, 112, 32  896         ['input_8[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_432 (Batch  (None, 112, 112, 32  128        ['conv2d_515[0][0]']             
 Normalization)                 )                                                           